In [1]:
#%load_ext watermark
#%watermark -v -p numpy,sklearn,scipy,matplotlib,tensorflow

***12장 – 분산 텐서플로***


목적
- 대규모 NN 을 여러 장치에 병렬 / 분산 실행하여 수행 시간을 절약한다.

Tensorflow 분산 처리의 장점
- 계산 그래프의 여러 장치 / 머신 분할방법 제어 가능
- 다양한 방식의 연산 병렬화 및 동기화 가능

병렬 수행의 실사례
- 신경망 병렬 수행
- 모델 세밀 튜닝을 위해 큰 하이퍼파라미터 공간을 탐색
- 대규모 NN 의 앙상블 수행

**12.1 단일 머신의 다중 장치**

- 단일 머신에 GPU 추가
- 다중 머신의 경우 네트워크 비용으로 인해 더 비효율적일 수 있음

*12.1.1 설치*

- 그래픽 카드 호환성 확인 : https://developer.nvidia.com/cuda-gpus
- CUDA : gpu 를 통한 명시적 computing 을 가능하게 하는 library
- cuDNN : DNN 을 위한 기초적 GPU 가속 library
 - activation function, normalize, forward / back propagation, pooling 등
- CUDA, cuDNN 바이너리 다운로드시 nvidia 개발자 계정이 필요 (https://developer.nvidia.com/)


* 본 실습에서는 gcp 를 활용

In [2]:
!nvidia-smi

Tue Jul 31 07:48:45 2018       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 396.37                 Driver Version: 396.37                    |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 00000000:00:04.0 Off |                    0 |
| N/A   73C    P0    88W / 149W |     11MiB / 11441MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
|   1  Tesla K80           Off  | 00000000:00:05.0 Off |                    0 |
| N/A   74C    P0    87W / 149W |     11MiB / 11441MiB |      9%      Default |
+-------------------------------+----------------------+----------------------+
        

- pyenv, anaconda 등 가상 환경을 사용하고 잇다면 적절한 환경으로 활성화

# 설정

파이썬 2와 3을 모두 지원합니다. 공통 모듈을 임포트하고 맷플롯립 그림이 노트북 안에 포함되도록 설정하고 생성한 그림을 저장하기 위한 함수를 준비합니다:

In [3]:
# 파이썬 2와 파이썬 3 지원
from __future__ import division, print_function, unicode_literals

# 공통
import numpy as np
import os

# 일관된 출력을 위해 유사난수 초기화
def reset_graph(seed=42):
    tf.reset_default_graph()
    tf.set_random_seed(seed)
    np.random.seed(seed)

# 맷플롯립 설정
%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt
plt.rcParams['axes.labelsize'] = 14
plt.rcParams['xtick.labelsize'] = 12
plt.rcParams['ytick.labelsize'] = 12

# 그림을 저장할 폴더
PROJECT_ROOT_DIR = "."
CHAPTER_ID = "distributed"

def save_fig(fig_id, tight_layout=True):
    path = os.path.join(PROJECT_ROOT_DIR, "images", CHAPTER_ID, fig_id + ".png")
    if tight_layout:
        plt.tight_layout()
    plt.savefig(path, format='png', dpi=300)

# 로컬 서버

In [4]:
import tensorflow as tf

/usr/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/usr/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/usr/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)


In [5]:
c = tf.constant("Hello distributed TensorFlow!")
server = tf.train.Server.create_local_server()

In [6]:
with tf.Session(server.target) as sess:
    print(sess.run(c))

b'Hello distributed TensorFlow!'


- gpu 초기화 및 정보 로그가 나온다면 성공

[I 12:02:07.853 NotebookApp] Adapting to protocol v5.1 for kernel 11e846df-9d53-4d1a-8be6-0622be66099f
2018-07-25 12:02:43.034875: I tensorflow/core/platform/cpu_feature_guard.cc:141] Your CPU supports instructions tha
t this TensorFlow binary was not compiled to use: AVX2 FMA
2018-07-25 12:02:43.140845: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:897] successful NUMA node read f
rom SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2018-07-25 12:02:43.141396: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1392] Found device 0 with properties
: 
name: Tesla K80 major: 3 minor: 7 memoryClockRate(GHz): 0.8235
pciBusID: 0000:00:04.0
totalMemory: 11.17GiB freeMemory: 11.10GiB
2018-07-25 12:02:43.222979: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:897] successful NUMA node read f
rom SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2018-07-25 12:02:43.223631: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1392] Found device 1 with properties
: 
name: Tesla K80 major: 3 minor: 7 memoryClockRate(GHz): 0.8235
pciBusID: 0000:00:05.0
totalMemory: 11.17GiB freeMemory: 11.10GiB
2018-07-25 12:02:43.223833: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1471] Adding visible gpu devices: 0,
 1
2018-07-25 12:02:43.774320: I tensorflow/core/common_runtime/gpu/gpu_device.cc:952] Device interconnect StreamExecu
tor with strength 1 edge matrix:
2018-07-25 12:02:43.774378: I tensorflow/core/common_runtime/gpu/gpu_device.cc:958]      0 1 
2018-07-25 12:02:43.774403: I tensorflow/core/common_runtime/gpu/gpu_device.cc:971] 0:   N Y 
2018-07-25 12:02:43.774408: I tensorflow/core/common_runtime/gpu/gpu_device.cc:971] 1:   Y N 
2018-07-25 12:02:43.774839: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1084] Created TensorFlow device (/jo
b:local/replica:0/task:0/device:GPU:0 with 10761 MB memory) -> physical GPU (device: 0, name: Tesla K80, pci bus id
: 0000:00:04.0, compute capability: 3.7)
2018-07-25 12:02:43.957038: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1084] Created TensorFlow device (/jo
b:local/replica:0/task:0/device:GPU:1 with 10761 MB memory) -> physical GPU (device: 1, name: Tesla K80, pci bus id
: 0000:00:05.0, compute capability: 3.7)
2018-07-25 12:02:44.140339: I tensorflow/core/distributed_runtime/rpc/grpc_channel.cc:215] Initialize GrpcChannelCa
che for job local -> {0 -> localhost:33667}
2018-07-25 12:02:44.141491: I tensorflow/core/distributed_runtime/rpc/grpc_server_lib.cc:334] Started server with t
arget: grpc://localhost:33667
2018-07-25 12:02:44.152483: I tensorflow/core/distributed_runtime/master_session.cc:1150] Start master session 6e81
0a0c2f63410d with config: 

*12.1.2 GPU RAM 관리*

- 단일 GPU 에 여러 프로그램 수행시 경우에 따라 OOM 발생할 수 있음

1. 별도 GPU 에 각각 수행

In [7]:
#CUDA_VISIBLE_DEVICES=0,1 python3 code.py - 복수 device ID 지정시 multi gpu 로 구동되는지 확인
#CUDA_VISIBLE_DEVICES="" 의 경우 CPU 로 수행됨

# https://stackoverflow.com/questions/37893755/tensorflow-set-cuda-visible-devices-within-jupyter
# http://dongjinlee.tistory.com/entry/%EC%84%A0%ED%83%9D%ED%95%9C-GPU%EC%97%90%EB%A7%8C-%EB%A9%94%EB%AA%A8%EB%A6%AC-%ED%95%A0%EB%8B%B9%ED%95%98%EB%8A%94-%EB%B0%A9%EB%B2%95
#import os
#os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"   # see issue #152
#os.environ["CUDA_VISIBLE_DEVICES"]="0"
# do some tensor

2. GPU 메모리 일부만을 사용하도록 강제

In [8]:
config = tf.ConfigProto()
config.gpu_options.per_process_gpu_memory_fraction = 0.4
session = tf.Session(config=config)

In [9]:
!nvidia-smi

Tue Jul 31 07:48:48 2018       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 396.37                 Driver Version: 396.37                    |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 00000000:00:04.0 Off |                    0 |
| N/A   73C    P0    80W / 149W |  10876MiB / 11441MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
|   1  Tesla K80           Off  | 00000000:00:05.0 Off |                    0 |
| N/A   74C    P0    76W / 149W |  10876MiB / 11441MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
        

3. 필요할 때 메모리를 확장 할당하는 옵션 활용
- 비활용 메모리 반납에 시간이 소요되므로(memory fragmentation 을 피하기 위해) 효용성이 낮음

In [10]:
config = tf.ConfigProto();
config.gpu_options.allow_growth = True
session = tf.Session(config=config)

*12.1.3 장치에 연산 배치하기*

dynamic placer
- 사용자 지정 배치 규칙에 비해 좋은 효율성을 보이고 있지 않다고 함.
- G사 내부용으로 현재 오픈소스화 되어 잇지 않음.

1) simple placer
* computation graph 수행시 device 에 배치되지 않는 node 를 평가할 때 사용


분배 규칙
- node 가 이미 배치되어 있는 장비는 해당 node를 그대로 둠
- 사용자가 node 를 어떤 장치에 할당햇다면 placer 가 node 를 배치
- GPU #0 이 기본, GPU 가 소진되면 CPU 로 전환

2) 배치 로깅

- placer 가 node 를 배치할 시점에 메세지를 기록하는 옵션
- TODO : 출력되는 log 에 대한 부연 설명

In [11]:
with tf.device("/cpu:0"):
    a = tf.Variable(3.0)
    b = tf.constant(4.0)

c = a * b

config = tf.ConfigProto()
config.log_device_placement = True
sess = tf.Session(config=config)

In [12]:
a.initializer.run(session=sess)

Variable: (VariableV2): /job:localhost/replica:0/task:0/device:CPU:
0
2018-07-30 21:38:01.248833: I tensorflow/core/common_runtime/placer.cc:886] Variable: (VariableV2)/job:l
ocalhost/replica:0/task:0/device:CPU:0
Variable/Assign: (Assign): /job:localhost/replica:0/task:0/device:CPU:0
2018-07-30 21:38:01.248857: I tensorflow/core/common_runtime/placer.cc:886] Variable/Assign: (Assign)/jo
b:localhost/replica:0/task:0/device:CPU:0
Variable/read: (Identity): /job:localhost/replica:0/task:0/device:CPU:0
2018-07-30 21:38:01.248876: I tensorflow/core/common_runtime/placer.cc:886] Variable/read: (Identity)/jo
b:localhost/replica:0/task:0/device:CPU:0
mul: (Mul): /job:localhost/replica:0/task:0/device:GPU:0
2018-07-30 21:38:01.248892: I tensorflow/core/common_runtime/placer.cc:886] mul: (Mul)/job:localhost/rep
lica:0/task:0/device:GPU:0
Const: (Const): /job:localhost/replica:0/task:0/device:CPU:0
2018-07-30 21:38:01.248903: I tensorflow/core/common_runtime/placer.cc:886] Const: (Const)/job:localhost
/replica:0/task:0/device:CPU:0
Variable/initial_value: (Const): /job:localhost/replica:0/task:0/device:CPU:0
2018-07-30 21:38:01.248915: I tensorflow/core/common_runtime/placer.cc:886] Variable/initial_value: (Con
st)/job:localhost/replica:0/task:0/device:CPU:0
Const_1: (Const): /job:localhost/replica:0/task:0/device:CPU:0
2018-07-30 21:38:01.248947: I tensorflow/core/common_runtime/placer.cc:886] Const_1: (Const)/job:localho
st/replica:0/task:0/device:CPU:0

In [13]:
sess.run(c)

12.0

3) 동적 배치 함수

- tf.device 함수 파라미터로 조건 함수를 넣을 수 있음

In [14]:
reset_graph()

def variables_on_cpu(op):
    if(op.type == "Variable"):
        return "/cpu:0"
    else:
        return "/gpu:0"

with tf.device(variables_on_cpu):
    a = tf.Variable(3.0)
    b = tf.constant(4.0)
    c = a * b

4) 연산과 커널

- CPU, GPU 장치별로 지원가능한 연산이 구분되어 잇음
- 커널 : 장치에 맞는 구현
- 예) integer Variable 은 GPU 에서 지원하지 않음 - 효율성 문제로 강제 미지원
- 하시 예시의 경우, dtype=tf.float32 로 명시하지 않으면 숫자 notation 을 통해 정수로 임의판단함

In [15]:
reset_graph()

with tf.device("/gpu:0"):
    a = tf.Variable(3) 
    #a = tf.Variable(3, dtype=tf.float32) 

with tf.Session() as sess:
    try:
        sess.run(a.initializer) # error
    except Exception as e:
        print(e)

Cannot assign a device for operation 'Variable': Could not satisfy explicit device specification '/device:GPU:0' because no supported kernel for GPU devices is available.
Colocation Debug Info:
Colocation group had the following types and devices: 
Assign: CPU 
Identity: GPU CPU 
VariableV2: CPU 

Colocation members and user-requested devices:
  Variable (VariableV2) /device:GPU:0
  Variable/Assign (Assign) /device:GPU:0
  Variable/read (Identity) /device:GPU:0

Registered kernels:
  device='CPU'
  device='GPU'; dtype in [DT_INT64]
  device='GPU'; dtype in [DT_DOUBLE]
  device='GPU'; dtype in [DT_FLOAT]
  device='GPU'; dtype in [DT_HALF]

	 [[Node: Variable = VariableV2[container="", dtype=DT_INT32, shape=[], shared_name="", _device="/device:GPU:0"]()]]

Caused by op 'Variable', defined at:
  File "/usr/lib/python3.5/runpy.py", line 184, in _run_module_as_main
    "__main__", mod_spec)
  File "/usr/lib/python3.5/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/usr/

5) 간접 배치

- 해당 커널이 없는 device 에 할당된 연산을, 해당 커널을 가진 임의 device 에 할당하는 옵션
- 묵시적 할당

In [16]:
reset_graph()

with tf.device("/gpu:0"):
    a = tf.Variable(3)
    
config = tf.ConfigProto()
config.log_device_placement = True
config.allow_soft_placement = True
session = tf.Session(config=config)
session.run(a.initializer) # /cpu:0

Variable: (VariableV2): /job:localhost/replica:0/task:0/device:CPU:0
2018-07-30 21:39:03.847717: I tensorflow/core/common_runtime/placer.cc:886] Variable: (VariableV2)/job:localhost/replica:0/task:0/device:CPU:0
Variable/Assign: (Assign): /job:localhost/replica:0/task:0/device:CPU:0
2018-07-30 21:39:03.847749: I tensorflow/core/common_runtime/placer.cc:886] Variable/Assign: (Assign)/job:localhost/replica:0/task:0/device:CPU:0
Variable/read: (Identity): /job:localhost/replica:0/task:0/device:CPU:0
2018-07-30 21:39:03.847778: I tensorflow/core/common_runtime/placer.cc:886] Variable/read: (Identity)/job:localhost/replica:0/task:0/device:CPU:0
Variable/initial_value: (Const): /job:localhost/replica:0/task:0/device:GPU:0
2018-07-30 21:39:03.847799: I tensorflow/core/common_runtime/placer.cc:886] Variable/initial_value: (Const)/job:localhost/replica:0/task:0/device:GPU:0

*12.1.4 병렬 실행*

CPU

1) queueing
- nn graph 실행시, 먼저 평가할 연산을 찾은 뒤 관련 연산들의 의존도를 측정
- 의존성이 전혀 없는 node 들을 할당된 장치의 evaluation queue 에 추가
- 하나의 연산이 평가되면 다른 모든 연산의 dependency counter 가 감소
- 그 후 dependency counter 가 0 이 되는 연산이 추가로 장치의 evaluation queue 에 추가
- 필요한 모든 Node 가 평가되면 output 을 return

2) pooling
- cpu 의 evaluation queue 에 있는 연산은 inter-op thread pool 로 이동
- 각 상황에 맞도록 병렬 처리  
 - multi-core hardware 를 이용
 - multithread cpu 커널 
  - 여러개의 부분연산으로 분리하여 다른 evaluation queue 에 배치
  - 상기 evaluation queue 에 배치된 연산은 (모든 multithread cpu 커널이 공유하는)intra-op thread pool 로 이동

- inter / intra-op thread pool 의 thread 수는 옵션으로 조정가능 - default 0 (모든 코어 사용)
 - 현재 CPU 특정 코어 지정이 불가하기 때문에, 옵션값을 CPU 코어 수 보다 적에 부여해야함

GPU
- GPU 상의 evaluation queue 연산들은 순서대로 평가됨
- 대다수의 연산에 대한 CUDA / cuDNN 기반 multithread GPU 커널 존재



<img src="fig12-5.png">

*12.1.5 제어 의존성*

- 의존 연산이 모두 수행되엇음에도, 효율을 위해 evaluation 을 가급적 delay 할 경우
-- 다량의 메모리 점유, 다수의 external I/O 발생 등
- 다른 연산을 병렬 처리 하며 순차적 실행




In [17]:
a = tf.constant(1.0)
b = a + 2.0

with tf.control_dependencies([a,b]):
    x = tf.constant(3.0)
    y = tf.constant(4.0)
    
z = x + y # z 도 a, b 의 evaluation 을 기다리는 의존성이 발생

**12.2 다중 머신의 다중 장치**

- task : 하나 이상의 텐서플로 서버로 구성
- job : 각기 이름이 부여된 task group
- cluster : task 라고 불리는 하나 이상의 텐서플로 서버로 구성


<img src="fig12-6.png">

# 클러스터

In [18]:
# import tensorflow as tf
# cluster_spec = tf.train.ClusterSpec({
#     "ps": [
#         "machine-a-host:2221",  # /job:ps/task:0
#     ],
#     "worker": [
#         "machine-a-host:2222",  # /job:worker/task:0 
#         "machine-b-host:2222",  # /job:worker/task:1
#     ]})

# # 동일 머신에서 여러 task 수행은 가능하나 비추천. 각 GPU RAM 점유를 수동으로 조정해줘야함
# task_ps0 = tf.train.Server(cluster_spec, job_name="ps", task_index=0)
# task_ps1 = tf.train.Server(cluster_spec, job_name="ps", task_index=1)
# ...

# #server.join() # 원격 클러스터를 호출하여 수행된 서버가 종료될 때까지 기다리도록 메인 스레드를 블록(명시적 서버 종료 방법은 없음)

In [19]:
# 클러스터에 사용할 원격 서버의 ip 기재
# 한 장비(ip)에 port 는 겹치면 정상 동작하지 않는다.
ps_task0_ip = "10.138.0.4:3333"
ps_task1_ip = "10.138.0.4:3334"
worker_task0_ip = "10.138.0.2:2221"
worker_task1_ip = "10.138.0.4:2222"

In [20]:
cluster_mgpu = tf.train.ClusterSpec({
    "ps": [
        ps_task0_ip,  # /job:ps/task:0
        ps_task1_ip,  # /job:ps/task:1
    ],
    "worker": [
        worker_task0_ip,  # /job:worker/task:0
        worker_task1_ip,  # /job:worker/task:1
    ]}
)

task_ps0 = tf.train.Server(cluster_mgpu, job_name="ps", task_index=0)
task_ps1 = tf.train.Server(cluster_mgpu, job_name="ps", task_index=1)

task_worker0 = tf.train.Server(cluster_mgpu, job_name="worker", task_index=0)
task_worker1 = tf.train.Server(cluster_mgpu, job_name="worker", task_index=1)

*12.2.1 세션 열기*

- 각 클러스터에서 tf.train.Server 로 클러스터를 띄운 후, 한 머신의 프로세스의 클라이언트에서 다른 모든 서버에 대해 세션을 열 수 있음

*12.2.2 마스터와 워커 서비스*

- Client / Server 는 gRPC 로 통신함
 - 적절한 통신 포트를 방화벽에서 열어주어야 함

- 텐서플로 서버는 기본적으로 마스터, 워커 서비스를 제공함
 - 마스터 : 클라이언트가 세션을 열고 그래플르 실행할 수 잇게 해줌
 - 워커 : 하나의 서버에서 graph 실행을 담당하는 RPC 서비스
    
- 유연성 제공
- 1 Client 가 n Server 에 접속하는 각각 session 오픈 가능
- task 마다 1 client 실행
- 1 client 로 여러 task 제어



*12.2.3 여러 태스크에 연산 할당하기*

- job 이름, task 번호, 장치 유형/번호 지정하여 연산 할당 가능
- 장치 유형/번호가 지정되지 않으면 해당 task 의 기본 장치 사용


In [21]:
reset_graph()

with tf.device("/job:ps"):
    a = tf.Variable(1, name="a")
    d = tf.Variable(3.0, name="d")

with tf.device("/job:ps/task:1"):
    e = tf.Variable(5.1, name="e")

with tf.device("/job:worker/task:0/gpu:0"):
    b = a + 2

with tf.device("/job:worker/task:1/gpu:1"):
    c = a + b


config = tf.ConfigProto()
config.log_device_placement = True
    
# 해당 원격 텐서플로 서버에 대한 세션을 오픈, c 를 evaluate 하라는 명령을 전달
# 해당 외부 장비의 기본장치(GPU) 에 배치 후 수행, 결과 반환
#with tf.Session(target="grpc://10.138.0.2:2221", config=config) as sess: 
with tf.Session(target="grpc://" + worker_task0_ip, config=config) as sess: 
    sess.run(a.initializer)
    print(c.eval())

4


*12.2.4 여러 대의 파라미터 서버에 변수를 나누어 분산하기*

- 다량의 파라미터가 있는 대규모 모델의 경우, 서버 한 대로 IO가 몰리지 않게 여러 서버에 분산해둠
- 별도 명시설정없이 모든 task 에 round-robin 할당해주는 방법 제공
- 대체로 파라미터 서버는 파라미터 저장 / 송수신 용도로 사용되고 무거운 연산을 수행하지 않게 구성

In [22]:
reset_graph()

with tf.device(tf.train.replica_device_setter( # cluster=cluster_spec 처럼 클러스터 명세를 전달해도 ps_tasks 를 찾아서 이용함
        ps_tasks=2,
        ps_device="/job:ps",
        worker_device="/job:worker")):
    v1 = tf.Variable(1.0, name="v1")  # /job:ps/task:0 (defaults to /cpu:0) 에 할당
    v2 = tf.Variable(2.0, name="v2")  # /job:ps/task:1 (defaults to /cpu:0) 에 할당
    v3 = tf.Variable(3.0, name="v3")  # /job:ps/task:0 (defaults to /cpu:0) 에 할당
    s = v1 + v2            # /job:worker (defaults to task:0/cpu:0) 에 할당
    with tf.device("/task:1"):
        p1 = 2 * s         # /job:worker/task:1 (defaults to /cpu:0) 에 할당
        with tf.device("/cpu:0"):
            p2 = 3 * s     # /job:worker/task:1/cpu:0 에 할당

config = tf.ConfigProto()
config.log_device_placement = True

with tf.Session("grpc://" + worker_task0_ip, config=config) as sess:
    v1.initializer.run()

*12.2.5 리소스 컨테이너를 사용해 여러 세션에서 상태 공유하기*

- local session 은 서로의 상태를 공유할 수 없음
- distributed session 의 경우 variable 의 상태를 클러스터 내부의 resource container 로 관리함
 - 특정 클라이언트 세션 한 곳에서 새 변수를 생성하면 동일 클러스터 내의 다른 세션에서도 자동으로 사용 가능
 - 리소스 컨테이너는 master task 에서 관리 되는듯


<img src="fig12-7.png">

In [23]:
config = tf.ConfigProto()

with tf.container("scope01"):
    x = tf.Variable(0.0, name="x")
    increment_x = tf.assign(x, x+1)

with tf.container("scope02"):
    x2 = tf.Variable(0.0, name="x")
    increment_x2 = tf.assign(x2, x2+1)

In [24]:
#tf.Session.reset("grpc://10.138.0.2:2221", ["scope01"])
#tf.Session.reset("grpc://10.138.0.2:2221", ["scope02"])

with tf.Session("grpc://" + worker_task0_ip, config=config) as sess:
    with tf.container("scope01"):
        sess.run(x.initializer)
        sess.run(increment_x)
        print(x.eval())
    with tf.container("scope02"):
        sess.run(x2.initializer)
        sess.run(increment_x2)
        sess.run(increment_x2)
        print(x2.eval())
    

1.0
2.0


*12.2.6 텐서플로 큐를 사용한 비동기 통신*

- 비동기 데이터 교환을 가능하게 함
 - 데이터 로더 -> 큐 -> 데이터 학습

- placeholder 를 사용해 클러스터에 데이터 주입

<img src="fig12-8.png">

- FIFOQueue
 - tuple 지원
- RandomShuffleQueue
- PaddingFIFOQueue

In [25]:
import tensorflow as tf
config = tf.ConfigProto()

# tuple
q = tf.FIFOQueue(capacity=10, dtypes=[tf.float32], shapes=[[2]], name="q", shared_name="shared_q")

training_instance = tf.placeholder(tf.float32, shape=(2))
enqueue = q.enqueue([training_instance]) 

training_instances = tf.placeholder(tf.float32, shape=(None, 2))
enqueue_many = q.enqueue_many([training_instances]) 

with tf.Session("grpc://" + worker_task0_ip, config=config) as sess:
    #sess.run(enqueue, feed_dict={training_instance: [1., 2.]})
    sess.run(enqueue_many, feed_dict={training_instances: [[1., 2.], [3., 4.]]})
    
    

In [26]:
config = tf.ConfigProto()

# tuple
q = tf.FIFOQueue(capacity=10, dtypes=[tf.float32], shapes=[[2]], name="q", shared_name="shared_q")

dequeue = q.dequeue() 

batch_size=2
dequeue_many = q.dequeue_many(batch_size) 

with tf.Session("grpc://" + worker_task0_ip, config=config) as sess:
    #sess.run(dequeue)
    print(sess.run(dequeue_many))
    
    

[[1. 2.]
 [3. 4.]]


In [27]:
reset_graph()

q = tf.FIFOQueue(capacity=10, dtypes=[tf.float32], shapes=[()])
v = tf.placeholder(tf.float32)
enqueue = q.enqueue([v])
dequeue = q.dequeue()
output = dequeue + 1

config = tf.ConfigProto()
config.operation_timeout_in_ms = 1000

with tf.Session(config=config) as sess:
    sess.run(enqueue, feed_dict={v: 1.0})
    sess.run(enqueue, feed_dict={v: 2.0})
    sess.run(enqueue, feed_dict={v: 3.0})
    print(sess.run(output))
    print(sess.run(output, feed_dict={dequeue: 5}))
    print(sess.run(output))
    print(sess.run(output))
    try:
        print(sess.run(output))
    except tf.errors.DeadlineExceededError as ex:
        print("dequeue 타임 아웃")

2.0
6.0
3.0
4.0
dequeue 타임 아웃


*12.2.7 그래프에서 직접 데이터 로드하기*

- 여러번 전송이 일어나서 대규모 환경에서는 비효율적
 - 파일IO > 클라이언트 > 마스터 태스크 > 데이터를 필요로하는 다른 태스크
- 여러 클라이언트가 동시레 한 자원(데이터) 을 사용할때

1) 데이터를 변수에 프리로드하기
- 훈련 데이터를 한번에 로드하여 변수에 할당(메모리 크기에 맞으면)
 - 클라이언트 > 클러스터 1회 전송


2) 그래프에서 직접 훈련 데이터 읽기
- 훈련 데이터가 메모리 크기에 안 맞다면 reader 활용
- 파일 시스템에서 직접 데이터를 읽음(클라이언트를 통하지 않고 가능)
- 지원 형식 : CSV, fixed length binary record, TFRecords



<img src="fig12-9.png">

In [28]:
reset_graph()

test_csv = open("my_test.csv", "w")
test_csv.write("x1, x2 , target\n")
test_csv.write("1.,, 0\n")
test_csv.write("4., 5. , 1\n")
test_csv.write("7., 8. , 0\n")
test_csv.close()

test_csv2 = open("my_test2.csv", "w")
test_csv2.write("x1, x2 , target\n")
test_csv2.write("1.,, 0\n")
test_csv2.write("14., 51. , 1\n")
test_csv2.write("17., 81. , 0\n")
test_csv2.close()

filename_queue = tf.FIFOQueue(capacity=10, dtypes=[tf.string], shapes=[()])
filename = tf.placeholder(tf.string)
enqueue_filename = filename_queue.enqueue([filename])
close_filename_queue = filename_queue.close()

reader = tf.TextLineReader(skip_header_lines=1)
key, value = reader.read(filename_queue)

x1, x2, target = tf.decode_csv(value, record_defaults=[[-1.], [-1.], [-1]])
features = tf.stack([x1, x2])

instance_queue = tf.RandomShuffleQueue(
    capacity=10, min_after_dequeue=2,
    dtypes=[tf.float32, tf.int32], shapes=[[2],[]],
    name="instance_q", shared_name="shared_instance_q")
enqueue_instance = instance_queue.enqueue([features, target])
close_instance_queue = instance_queue.close()

minibatch_instances, minibatch_targets = instance_queue.dequeue_up_to(2)

with tf.Session() as sess:
    sess.run(enqueue_filename, feed_dict={filename: "my_test.csv"})
    sess.run(enqueue_filename, feed_dict={filename: "my_test2.csv"})
    sess.run(close_filename_queue)
    try:
        while True:
            sess.run(enqueue_instance)
    except tf.errors.OutOfRangeError as ex:
        print("더 이상 읽을 파일이 없습니다")
    sess.run(close_instance_queue)
    try:
        while True:
            print(sess.run([minibatch_instances, minibatch_targets]))
    except tf.errors.OutOfRangeError as ex:
        print("더 이상 훈련 샘플이 없습니다")

더 이상 읽을 파일이 없습니다
[array([[ 4.,  5.],
       [17., 81.]], dtype=float32), array([1, 0], dtype=int32)]
[array([[14., 51.],
       [ 1., -1.]], dtype=float32), array([1, 0], dtype=int32)]
[array([[ 1., -1.],
       [ 7.,  8.]], dtype=float32), array([0, 0], dtype=int32)]
더 이상 훈련 샘플이 없습니다


# 리더 (Reader) - 예전 방법

In [29]:
reset_graph()

default1 = tf.constant([5.])
default2 = tf.constant([6])
default3 = tf.constant([7])
dec = tf.decode_csv(tf.constant("1.,,44"),
                    record_defaults=[default1, default2, default3])
with tf.Session() as sess:
    print(sess.run(dec))

[1.0, 6, 44]


In [30]:
#coord = tf.train.Coordinator()
#threads = tf.train.start_queue_runners(coord=coord)
#filename_queue = tf.train.string_input_producer(["test.csv"])
#coord.request_stop()
#coord.join(threads)

3) Coordinator 와 QueueRunner 를 사용하는 멀티스레드 리더

- 여러 스레드가 여러 리더로 여러 파일을 동시에 read 할 목적
- 스레드 구현 및 제어 관리를 신경쓰지 않고 사용 가능

<img src="fig12-10.png">

In [31]:
reset_graph()

filename_queue = tf.FIFOQueue(capacity=10, dtypes=[tf.string], shapes=[()])
filename = tf.placeholder(tf.string)
enqueue_filename = filename_queue.enqueue([filename])
close_filename_queue = filename_queue.close()

reader = tf.TextLineReader(skip_header_lines=1)
key, value = reader.read(filename_queue)

x1, x2, target = tf.decode_csv(value, record_defaults=[[-1.], [-1.], [-1]])
features = tf.stack([x1, x2])

instance_queue = tf.RandomShuffleQueue(
    capacity=10, min_after_dequeue=2,
    dtypes=[tf.float32, tf.int32], shapes=[[2],[]],
    name="instance_q", shared_name="shared_instance_q")
enqueue_instance = instance_queue.enqueue([features, target])
close_instance_queue = instance_queue.close()

minibatch_instances, minibatch_targets = instance_queue.dequeue_up_to(2)

n_threads = 5
queue_runner = tf.train.QueueRunner(instance_queue, [enqueue_instance] * n_threads)
coord = tf.train.Coordinator()

with tf.Session() as sess:
    sess.run(enqueue_filename, feed_dict={filename: "my_test.csv"})
    sess.run(enqueue_filename, feed_dict={filename: "my_test2.csv"})    
    sess.run(close_filename_queue)
    enqueue_threads = queue_runner.create_threads(sess, coord=coord, start=True)
    try:
        while True:
            print(sess.run([minibatch_instances, minibatch_targets]))
    except tf.errors.OutOfRangeError as ex:
        print("더 이상 훈련 샘플이 없습니다")

[array([[ 7.,  8.],
       [ 1., -1.]], dtype=float32), array([0, 0], dtype=int32)]
[array([[17., 81.],
       [ 1., -1.]], dtype=float32), array([0, 0], dtype=int32)]
[array([[14., 51.],
       [ 4.,  5.]], dtype=float32), array([1, 1], dtype=int32)]
더 이상 훈련 샘플이 없습니다


In [32]:
reset_graph()

def read_and_push_instance(filename_queue, instance_queue):
    reader = tf.TextLineReader(skip_header_lines=1)
    key, value = reader.read(filename_queue)
    x1, x2, target = tf.decode_csv(value, record_defaults=[[-1.], [-1.], [-1]])
    features = tf.stack([x1, x2])
    enqueue_instance = instance_queue.enqueue([features, target])
    return enqueue_instance

filename_queue = tf.FIFOQueue(capacity=10, dtypes=[tf.string], shapes=[()])
filename = tf.placeholder(tf.string)
enqueue_filename = filename_queue.enqueue([filename])
close_filename_queue = filename_queue.close()

instance_queue = tf.RandomShuffleQueue(
    capacity=10, min_after_dequeue=2,
    dtypes=[tf.float32, tf.int32], shapes=[[2],[]],
    name="instance_q", shared_name="shared_instance_q")

minibatch_instances, minibatch_targets = instance_queue.dequeue_up_to(2)

read_and_enqueue_ops = [read_and_push_instance(filename_queue, instance_queue) for i in range(5)]
queue_runner = tf.train.QueueRunner(instance_queue, read_and_enqueue_ops)

with tf.Session() as sess:
    sess.run(enqueue_filename, feed_dict={filename: "my_test.csv"})
    sess.run(enqueue_filename, feed_dict={filename: "my_test2.csv"})
    sess.run(close_filename_queue)
    coord = tf.train.Coordinator()
    enqueue_threads = queue_runner.create_threads(sess, coord=coord, start=True)
    try:
        while True:
            print(sess.run([minibatch_instances, minibatch_targets]))
    except tf.errors.OutOfRangeError as ex:
        print("더 이상 훈련 샘플이 없습니다")

[array([[ 1., -1.],
       [ 4.,  5.]], dtype=float32), array([0, 1], dtype=int32)]
[array([[14., 51.],
       [ 1., -1.]], dtype=float32), array([1, 0], dtype=int32)]
[array([[17., 81.],
       [ 7.,  8.]], dtype=float32), array([0, 0], dtype=int32)]
더 이상 훈련 샘플이 없습니다


# Data API

텐서플로 1.4에서 소개된 Data API를 사용하면 손쉽게 데이터를 효율적으로 읽을 수 있습니다.

In [33]:
tf.reset_default_graph()

0에서 9까지 정수를 세 번 반복한 간단한 데이터셋을 일곱 개씩 배치로 만들어 시작해 보죠:

In [34]:
dataset = tf.data.Dataset.from_tensor_slices(np.arange(10))
dataset = dataset.repeat(3).batch(7)

첫 번째 줄은 0에서 9까지 정수를 담은 데이터셋을 만듭니다. 두 번째 줄은 이 데이터셋의 원소를 세 번 반복하고 일곱 개씩 담은 새로운 데이터셋을 만듭니다. 위에서 볼 수 있듯이 원본 데이터셋에서 여러 변환 메서드를 연결하여 호출하여 적용했습니다.

그다음, 데이터셋을 한 번 순회하는 원-샷-이터레이터(one-shot-iterator)를 만들고, 다음 원소를 지칭하는 텐서를 얻기 위해 `get_next()` 메서드를 호출합니다.

In [35]:
iterator = dataset.make_one_shot_iterator()
next_element = iterator.get_next()

`next_element`를 반복적으로 평가해서 데이터셋을 순회해 보죠. 원소가 별로 없기 때문에 `OutOfRangeError`가 발생합니다:

In [36]:
with tf.Session() as sess:
    try:
        while True:
            print(next_element.eval())
    except tf.errors.OutOfRangeError:
        print("완료")

[0 1 2 3 4 5 6]
[7 8 9 0 1 2 3]
[4 5 6 7 8 9 0]
[1 2 3 4 5 6 7]
[8 9]
완료


좋네요! 잘 작동합니다.

늘 그렇듯이 텐서는 그래프를 실행(`sess.run()`)할 때마다 한 번만 평가된다는 것을 기억하세요. `next_element`에 의존하는 텐서를 여러개 평가하더라도 한 번만 평가됩니다. 또한 `next_element`를 동시에 두 번 실행해도 마찬가지입니다:

In [37]:
with tf.Session() as sess:
    try:
        while True:
            print(sess.run([next_element, next_element]))
    except tf.errors.OutOfRangeError:
        print("완료")

[array([0, 1, 2, 3, 4, 5, 6]), array([0, 1, 2, 3, 4, 5, 6])]
[array([7, 8, 9, 0, 1, 2, 3]), array([7, 8, 9, 0, 1, 2, 3])]
[array([4, 5, 6, 7, 8, 9, 0]), array([4, 5, 6, 7, 8, 9, 0])]
[array([1, 2, 3, 4, 5, 6, 7]), array([1, 2, 3, 4, 5, 6, 7])]
[array([8, 9]), array([8, 9])]
완료


`interleave()` 메서드는 강력하지만 처음에는 이해하기 좀 어렵습니다. 예제를 통해 이해하는 것이 가장 좋습니다:

In [38]:
tf.reset_default_graph()

In [39]:
dataset = tf.data.Dataset.from_tensor_slices(np.arange(10))
dataset = dataset.repeat(3).batch(7)
dataset = dataset.interleave(
    lambda v: tf.data.Dataset.from_tensor_slices(v),
    cycle_length=3,
    block_length=2)
iterator = dataset.make_one_shot_iterator()
next_element = iterator.get_next()

In [40]:
with tf.Session() as sess:
    try:
        while True:
            print(next_element.eval(), end=",")
    except tf.errors.OutOfRangeError:
        print("완료")

0,1,7,8,4,5,2,3,9,0,6,7,4,5,1,2,8,9,6,3,0,1,2,8,9,3,4,5,6,7,완료


`cycle_length=3`이므로 새로운 데이터셋은 이전 데이터셋에서 세 개의 원소를 추출합니다. 즉 `[0,1,2,3,4,5,6]`, `[7,8,9,0,1,2,3]`, `[4,5,6,7,8,9,0]` 입니다. 그다음 원소마다 하나의 데이터셋을 만들기 위해 람다(lambda) 함수를 호출합니다. `Dataset.from_tensor_slices()`를 사용했기 때문에 각 데이터셋은 차례대로 원소를 반환합니다. 다음 이 세 개의 데이터셋에서 각각 두 개의 아이템(`block_length=2`이므로)을 추출합니다. 세 개의 데이터셋의 아이템이 모두 소진될 때까지 반복됩니다. 즉 0,1 (첫 번째에서), 7,8 (두 번째에서), 4,5 (세 번째에서), 2,3 (첫 번째에서), 9,0 (두 번째에서) 등과 같은 식으로 8,9 (세 번째에서), 6 (첫 번째에서), 3 (두 번째에서), 0 (세 번째에서)까지 진행됩니다. 그다음에 원본 데이터셋에서 다음 번 세 개의 원소를 추출하려고 합니다. 하지만 두 개만 남아 있습니다. `[1,2,3,4,5,6,7]`와 `[8,9]` 입니다. 다시 이 원소로부터 데이터셋을 만들고 이 데이텃세의 아이템이 모두 소진될 때까지 두 개의 아이템을 추출합니다. 1,2 (첫 번째에서), 8,9 (두 번째에서), 3,4 (첫 번째에서), 5,6 (첫 번째에서), 7 (첫 번째에서)가 됩니다. 배열의 길이가 다르기 때문에 마지막에는 교대로 배치되지 않았습니다.

# 리더 (Reader) - 새로운 방법

`from_tensor_slices()`나 `from_tensor()`를 기반으로 한 원본 데이터셋을 사용하는 대신 리더 데이터셋을 사용할 수 있습니다. 복잡한 일들을 대부분 대신 처리해 줍니다(예를 들면, 스레드):

In [41]:
tf.reset_default_graph()

In [42]:
filenames = ["my_test.csv"]

In [43]:
dataset = tf.data.TextLineDataset(filenames)

각 줄을 어떻게 디코드해야 하는지는 알려 주어야 합니다:

In [44]:
def decode_csv_line(line):
    x1, x2, y = tf.decode_csv(
        line, record_defaults=[[-1.], [-1.], [-1.]])
    X = tf.stack([x1, x2])
    return X, y

그다음, 이 디코딩 함수를 `map()`을 사용하여 데이터셋에 있는 각 원소에 적용할 수 있습니다:

In [45]:
dataset = dataset.skip(1).map(decode_csv_line)

마지막으로 원-샷-이터레이터를 만들어 보죠:

In [46]:
it = dataset.make_one_shot_iterator()
X, y = it.get_next()

In [47]:
with tf.Session() as sess:
    try:
        while True:
            X_val, y_val = sess.run([X, y])
            print(X_val, y_val)
    except tf.errors.OutOfRangeError as ex:
        print("완료")

[ 1. -1.] 0.0
[4. 5.] 1.0
[7. 8.] 0.0
완료


**12.3 텐서플로 클러스터에서 신경망 병렬화하기**



*12.3.1 장치마나 하나의 신경망*

- 가장 단순함, 직관적
- 여러 하이퍼파라미터를 돌려보며 튜닝하는 목적
- realtime prediction 전용 환경으로도 좋음(gpu 를 증가시켜 scale-out 가능)

- tensorflow serving



<img src="fig12-11.png">

*12.3.2 그래프 내 복제와 그래프 간 복제*

- 여러 신경망을 분산배치하여 앙상블을 구성할 수 잇음
- 모든 신경망들의 prediction 을 모아서 앙상블의 prediction 이 가능
- 앙상블 구성방법
 - 그래프 내 복제 : 구현이 간단함






<img src="fig12-12.png">

 - 그래프 간 복제 : 구현이 복잡하지만, queue 통신 기반이기 때문에 앙상블의 안정성을 좀 더 보장해줌

<img src="fig12-13.png">

*12.3.3 모델 병렬화*

- 모델을 여러 부분으로 나누어 각 부분을 다른 장치에서 실행
- 신경망 모델 구조에 따라 효율성과 구현 난이도가 갈림



1) Fully connected

<img src="fig12-14.png">

2) Partialy connected

<img src="fig12-15.png">

3) Deep RNN

<img src="fig12-16.png">

*12.3.4 데이터 병렬화*

- 각 장치에 모델 복제 후 각각 다른 데이터(미니배치) 를 사용
- 훈련 후 발생하는 gradient 를 취합 후 모델 파라미터 업데이트
 - 동기 / 비동기

<img src="fig12-17.png">

1) 동기

- 모든 gradient 가 계산될때까지 대기한 후 평균을 계산
- 거의 같은 시점에 모든 복제모델에 파라미터가 업데이트 

2) 비동기

- gradient 가 계산될 시점에 그때그때 모델 파라미터 업데이트
- 훈련학습이 빨리되는 모델이 더 파라미터를 자주 업데이트하게 됨
- stale gradient
 - gradient 가 계산된 후 그 값으로 파라미터가 업데이트 되기 전에 다른 모델이 파라미터를 업데이트 한다면?

<img src="fig12-18.png">

3) 대역폭 포화

- gpu ram i/o 시간 > gpu 연산 계산시간(분할된)
 - 네트워크나 버스로?
- 모델이 규모가 작고 데이터 량이 많다면 1 gpu 1 machine 이 더 낫다고함

- 대역폭 감소 방안
 - 적은 수의 머신에 GPU 를 모아서 네트워크 통신 최소화
 - 여러 대의 파라미터 서버에 파라미터 분산
 - 모델 파라미터 정밀도 조정 (float32 -> float16)
 
4) 텐서플로 구현 

- 그래프 내 복제 / 그래프 간 복제, 동기 / 비동기 등 방법 선택
- 

# 연습문제 해답

https://github.com/ageron/handson-ml/issues/187

1. 텐서플로 프로그램 수행시 CUDA_ERROR_OUT_OF_MEMORY 오류가 발생하는 원인 및 해결책은 무엇인가?

2. (장치) 연산 할당 / 연산 배치 의 차이점.

3. GPU 버젼 텐서플로의 기본 배치로 실행하면 모든 연산이 GPU #0 에 배치되는가?

4. "/gpu:0" 에 할당된 Variable 을 "/gpu:1" 상의 연산이 사용 할 수 있는가?

    또는 "/cpu:0" 상의 연산이 사용 할 수 있는가? 다른 서버 장치에 할당된 연산은?


5. 같은 장치에 배치된 두 연산이 동시에 수행될 수 있는가?

6. 제어 의존성이 무엇인가? 언제 사용되는가?

7. 클러스터로 며칠간 DNN 을 훈련한 후 Saver 로 모델을 저장하지 않앗다면 모델은 날아간 것인가?

8. 클러스터에서 몇 개의 DNN 을 각기 다른 hparam 으로 훈련해 보라.
   cross validation 또는 validation set 을 통해 가장 좋은 모델 세 개를 선택하라.

9. 상기 선택된 모델 세 개로 앙상블을 구성하라. 개별 DNN 보다 성능이 더 나은가?

10. 그래프 간 복제, 비동기 업데이트를 통해 DNN 을 훈련 시켜보라. 동기 업데이트를 사용해 다시 훈련해보라.
    성능 및 훈련 소요 시간을 비교해보라.
    DNN 수직 분할 후 훈련시켰을 때도 추가로 비교해 보라.